# scikit-XCS Complete User Guide

Author: Robert Zhang - University of Pennsylvania, B.S.E Computer Science, B.S.E. Economics (SEAS '22, WH '22)

Advisor: Ryan Urbanowicz, PhD - University of Pennsylvania, Department of Biostatics, Epidemiology, and Informatics & Institue for Biomedical Informatics (IBI)

Special thanks to Will N. Browne (Victoria University of Wellingston), for his valuable input in the compilation of the scikit-XCS algorithm

Date: 04/30/2020

Package Requirements: (Python 3)
<ul>
    <li>pandas</li>
    <li>numpy</li>
    <li>scikit-learn</li>
</ul>

Notebook Requirements:
<ul>
    <li>scikit-XCS</li>
    <li>scikit-learn</li>
    <li>pandas</li>
    <li>mathplotlib</li>
</ul>

## Introduction
This notebook presents a complete user guide to the core sklearn-compatible scikit-XCS package as well as auxilliary helper objects included in the package. XCS is the most popular and best studied learning classifier system algorithm to date. In general, Learning Classifier Systems (LCSs) are a classification of Rule Based Machine Learning Algorithms that have been shown to perform well on problems involving high amounts of heterogeneity and epistasis. Well designed LCSs are also highly human interpretable. LCS variants have been shown to adeptly handle supervised and reinforced, classification and regression, online and offline learning problems, as well as missing or unbalanced data. These characteristics of versatility and interpretability give LCSs a wide range of potential applications, notably those in biomedicine. This package is still under active development and we encourage you to check back on this repository for updates.

This version of scikit-XCS is suitable for **single step, classification** problems. It has not yet been developed for multi-step reinforcement learning problems nor regression problems. Within these bounds however, scikit-XCS can be applied to almost any supervised classification data set and supports:

<ul>
    <li>Feature sets that are discrete/categorical, continuous-valued or a mix of both</li>
    <li>Data with missing values</li>
    <li>Binary Classification Problems (Binary Endpoints)</li>
    <li>Multi-class Classification Problems (Multi-class Endpoints)</li>
</ul>

## Notebook Organization

**Part 1: Loading Data**
<ul>
    <li> Dataset format requirements for scikit-XCS</li>
    <li> Method 1: Loading data using pandas</li>
    <li> Method 2: Loading data using StringEnumerator (additional data loading/transformation tool provided by scikit-XCS package)</li>
</ul>

**Part 2: Initializing XCS Estimator Object**
<ul>
    <li> Basic Initialization</li>
    <li> Overview of all scikit-XCS initialization parameters and default values</li>
</ul>

**Part 3: Model Training and Testing**
<ul>
    <li> model.fit(X,y) and how it works</li>
    <li> model.predict(X), model.predict_proba(X)</li>
    <li> model Cross Validation and Scoring</li>
    <li> ROC/PRC curves and AUC</li>
    <li> Auxilliary scikit-XCS methods
</ul>

**Part 4: Data Collection and Export**
<ul>
    <li> Exporting and Accessing Iteration Tracking Data </li>
    <li> Exporting and Accessing Rule Population Data </li>
</ul>

**Part 5: Population Reboot**


## Part 1: Loading Data

### Dataset Format Requirement for scikit-XCS

There are 4 requirements for the dataset:
<ul>
    <li>Data Attributes for all data instances are represented by an <b>nxm</b> numpy.ndarray, where <b>n</b> = # of instances and <b>m</b> = # of attributes for each instance</li>
    <li>Data Actions for all data instances are represented by a 1 dimensional numpy.ndarray of length <b>n</b></li>
    <li>Both the <b>nxm</b> attribute array and the length <b>n</b> action array are fully numeric. This means that every element in each array must be convertable to a float. String, boolean types are not permitted (of course, boolean datasets can be represented by binary 0s and 1s). It is important to mention that missing data in the attribute dataset is acceptable, if represented as a NaN type within the array. Missing data in the action array is not acceptable.</li>
    <li>As mentioned aboved, the problem must be a classification problem. This version does not support regression problems.</li>
</ul>

The package includes automated parameter checking that will raise an Exception at the start of training if the dataset does not meet the above requirements.

Below, we present 2 methods to load data of the correct format for the XCS model. The first is the commonly used pandas. The second is a custom, standalone method included with this package named StringEnumerator, that will make your life much easier in terms of ensuring data formatting requirements are met for the XCS model.

### Loading Data Method 1: Using pandas

There are many ways you can derive the attribute and phenotype array. One most common method is by using pandas via reading from a csv file, as the below code demonstrates with the a 6 bit multiplexer dataset:

In [1]:
import pandas as pd

#Read from CSV file
data = pd.read_csv("test/DataSets/Real/Multiplexer6.csv")

#Specify the dataset's action label
actionLabel = "class"

#Derive the attribute and action array using the action label
dataFeatures = data.drop(actionLabel,axis = 1).values
dataActions = data[actionLabel].values

#Optional: Retrieve the headers for each attribute as a length n array
dataHeaders = data.drop(actionLabel,axis=1).columns.values

print("Data Features")
print(dataFeatures)
print("\nData Actions")
print(dataActions)
print("\nData Headers")
print(dataHeaders)

Data Features
[[0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 1 1]
 [0 0 0 1 0 0]
 [0 0 0 1 0 1]
 [0 0 0 1 1 0]
 [0 0 0 1 1 1]
 [0 0 1 0 0 0]
 [0 0 1 0 0 1]
 [0 0 1 0 1 0]
 [0 0 1 0 1 1]
 [0 0 1 1 0 0]
 [0 0 1 1 0 1]
 [0 0 1 1 1 0]
 [0 0 1 1 1 1]
 [0 1 0 0 0 0]
 [0 1 0 0 0 1]
 [0 1 0 0 1 0]
 [0 1 0 0 1 1]
 [0 1 0 1 0 0]
 [0 1 0 1 0 1]
 [0 1 0 1 1 0]
 [0 1 0 1 1 1]
 [0 1 1 0 0 0]
 [0 1 1 0 0 1]
 [0 1 1 0 1 0]
 [0 1 1 0 1 1]
 [0 1 1 1 0 0]
 [0 1 1 1 0 1]
 [0 1 1 1 1 0]
 [0 1 1 1 1 1]
 [1 0 0 0 0 0]
 [1 0 0 0 0 1]
 [1 0 0 0 1 0]
 [1 0 0 0 1 1]
 [1 0 0 1 0 0]
 [1 0 0 1 0 1]
 [1 0 0 1 1 0]
 [1 0 0 1 1 1]
 [1 0 1 0 0 0]
 [1 0 1 0 0 1]
 [1 0 1 0 1 0]
 [1 0 1 0 1 1]
 [1 0 1 1 0 0]
 [1 0 1 1 0 1]
 [1 0 1 1 1 0]
 [1 0 1 1 1 1]
 [1 1 0 0 0 0]
 [1 1 0 0 0 1]
 [1 1 0 0 1 0]
 [1 1 0 0 1 1]
 [1 1 0 1 0 0]
 [1 1 0 1 0 1]
 [1 1 0 1 1 0]
 [1 1 0 1 1 1]
 [1 1 1 0 0 0]
 [1 1 1 0 0 1]
 [1 1 1 0 1 0]
 [1 1 1 0 1 1]
 [1 1 1 1 0 0]
 [1 1 1 1 0 1]
 [1 1 1 1 1 0]
 [1 1 1 1 1 1]]

Data Actions
[0 0 0 0 0 

Below is another code snippet that uses the same method, but with a dataset that contains missing values. Note that all missing values must be represented as a NaN type when used in the scikit-XCS model.

In [2]:
import pandas as pd

#Read from CSV file
data = pd.read_csv("test/DataSets/Tests/MissingFeatureData.csv")

#Specify the dataset's action label
actionLabel = "phenotype"

#Derive the attribute and action array using the action label
dataFeatures = data.drop(actionLabel,axis = 1).values
dataActions = data[actionLabel].values

#Optional: Retrieve the headers for each attribute as a length n array
dataHeaders = data.drop(actionLabel,axis=1).columns.values

print("Data Features")
print(dataFeatures)
print("\nData Actions")
print(dataActions)
print("\nData Headers")
print(dataHeaders)

Data Features
[[ 1. nan  1.  4.]
 [ 2.  0.  1. nan]
 [ 4. nan  1.  2.]
 [nan  1. nan  1.]
 [ 6. nan  1.  1.]]

Data Actions
[1 0 1 0 1]

Data Headers
['N1' 'N2' 'N3' 'N4']


### Loading Data Method 2: Using StringEnumerator

Given the 4 requirements for the attribute and phenotype array, not all datasets necessarily satisfy those requirements. For example, a dataset may contain an attribute with values that are colors "red", "green", "blue". Since the scikit-XCS model can only work with numeric data, this dataset would not meet the requirements. Alternatively, a dataset may, for some reason, have missing phenotype data. Since the scikit-XCS model requires that the phenotype array is complete, this dataset would also not meet the requirements.

To make things more convenient, included in this package is an auxilliary data type called **String Enumerator** that makes it easy to transform datasets that do not meet the requirements into datasets that do.

For example, take the below dummy dataset:

In [3]:
import pandas as pd

#Read from CSV file
data = pd.read_csv("test/DataSets/Tests/StringData2.csv")

#Specify the dataset's action label
actionLabel = "phenotype"

#Derive the attribute and phenotype array using the action label
dataFeatures = data.drop(actionLabel,axis = 1).values
dataActions = data[actionLabel].values

#Optional: Retrieve the headers for each attribute as a length n array
dataHeaders = data.drop(actionLabel,axis=1).columns.values

print("Data Features")
print(dataFeatures)
print("\nData Actions")
print(dataActions)
print("\nData Headers")
print(dataHeaders)

Data Features
[['male' 1.2 'young']
 ['female' 0.3 nan]
 ['female' -0.4 'old']
 [nan 0.0 'young']]

Data Actions
['china' nan 'china' 'russia']

Data Headers
['N1' 'N2' 'N3']


This dummy dataset does not come close to meeting the dataset requirements for training. There exist many string types in the attribute and phenotype array, as well as missing phenotype data. Aside from not meeting the requirements, the data headers and phenotype label are not too descriptive. Below StringEnumerator cleans this up.

In [4]:
from skXCS import StringEnumerator

#Initialize StringEnumerator object with csv filepath and class label.
converter = StringEnumerator("test/DataSets/Tests/StringData2.csv","phenotype")

#Change Header Names to be more descriptive using change_header_name(currentName,newName)
converter.change_header_name("N1","Gender")
converter.change_header_name("N2","Numeric")
converter.change_header_name("N3","Age")

#Change Phenotype Label to be more descriptive using change_class_name(newName)
converter.change_class_name("Country")

'''
Convert attributes to numeric data using either:
-add_attribute_converter_random(headerName):   Given an attribute name, randomly assigns each unique attribute value an
                                            integer value from 0 to n-1, where n = # of unique attribute values
                                            
-add_attribute_converter(headerName,array):   Given an attribute name, and an array of attribute values that will be
                                            converted, converter assigns each attribute value an integer value from
                                            0 to n-1 in the order of attribute values given in the array, where
                                            n = length of array. This can be useful for discrete attributes where the
                                            ordering of the attributes are important (ordinal values). For example,
                                            an attribute may have values "stage 1", "stage 2", "stage 3", "stage 4"
                                            to indicate stage of cancer, where the values are strings, but how they
                                            are enumerated is crucial.
'''
converter.add_attribute_converter_random("Gender")
converter.add_attribute_converter("Age",["old","young"])

'''
Convert phenotypes to numeric data using either
-add_class_converter(array)
-add_class_converter_random()
Same functionality as the attribute converters
'''
converter.add_class_converter_random()

'''
Convert all attributes using convert_all_attributes(). Note: this is NOT a standalone method. This method
just puts all of the changes made above "into stone". This method does not automatically convert anything for you. You
MUST call this method if data transformation operations were invoked (any of the class/attributeConverter methods
above), or you will not see any changes when you call get_params().
'''
converter.convert_all_attributes()

#Get arrays using get_params()
headers,classLabel,dataFeatures,dataPhenotypes = converter.get_params()

print("Data Features")
print(dataFeatures)
print("\nData Phenotypes")
print(dataPhenotypes)
print("\nData Headers")
print(headers)
print("\nClass Label")
print(classLabel)

Data Features
[[ 0.   1.2  1. ]
 [ 1.  -0.4  0. ]
 [ nan  0.   1. ]]

Data Phenotypes
[0. 0. 1.]

Data Headers
['Gender' 'Numeric' 'Age']

Class Label
Country


As you can see, the StringEnumerator mapped all string attributes and phenotypes into numeric types, deleted the instance that was missing phenotype data, and made the data headers and class label more descriptive. Now, the data features and phenotypes are ready for training.

The StringEnumerator has 2 additional methods not shown above that can also be powerful in data transformation:
<ul>
    <li>delete_attribute(headerName): deletes the specified attribute from all instances in the dataset</li>
    <li>delete_all_instances_without_header_data(headerName): there may be situations where you only want to use data instances that does not have missing data for a specific attribute. This method deletes all instances who's specified attribute is missing</li>
</ul>

Even if the dataset satisfies all requirements, StringEnumerator can be used as an easy way to get data in the correct format without needing to use pandas, as shown below:

In [5]:
converter = StringEnumerator("test/DataSets/Tests/MissingFeatureData.csv","phenotype")
headers,actionLabel,dataFeatures,dataActions = converter.get_params()

print("Data Features")
print(dataFeatures)
print("\nData Actions")
print(dataActions)
print("\nData Headers")
print(headers)

Data Features
[[ 1. nan  1.  4.]
 [ 2.  0.  1. nan]
 [ 4. nan  1.  2.]
 [nan  1. nan  1.]
 [ 6. nan  1.  1.]]

Data Actions
[1. 0. 1. 0. 1.]

Data Headers
['N1' 'N2' 'N3' 'N4']


Finally, StringEnumerator contains a print_invalid_attributes() method that will print out the names of any attribute with invalid typed data (including the phenotype). This can help you identify which attributes need to converted.

In [6]:
converter = StringEnumerator("test/DataSets/Tests/StringData2.csv","phenotype")
converter.print_invalid_attributes()

ALL INVALID ATTRIBUTES & THEIR DISTINCT VALUES
N1: male	female	
N3: young	old	
phenotype (the phenotype): china	russia	


All in all, StringEnumerator makes loading data in the correct format easy. It is intuitive to use, and if any invalid operation were done with it, built in checkers will raise contextually Exceptions to help you correct the error. We will be using it to load data for the rest of this user guide.

## Part 2: Initializing scikit-XCS Estimator Object

### Basic Initialization

In [7]:
from skXCS import XCS

model = XCS()

Doing the above will initialize an XCS estimator object using the default hyperparameters, that is now ready for training. However, in most cases, you would want to modify some hyperparameters to your needs. We list all of the tunable hyperparameters and their descriptions:

### Overview of all scikit-XCS initialization parameters and default values

| Parameter Name | Requirements | Description | Default Value |
| :--- | :--- | :--- | :--- |
| learning_iterations | non negative integer | The number of training cycles to run | 10000 |
| N | non negative integer | Maximum micropopulation size | 1000 |
| p_general | float from 0 - 1 | Probability of generalizing an allele during covering | 0.5 |
| p_crossover | float from 0 - 1 | The probability of applying crossover in an offspring classifier | 0.8 |
| p_mutation | float from 0 - 1 | The probability of mutating one allele and the action in an offspring classifier | 0.04 |
| p_explore | float from 0 - 1 | Probability of doing an explore cycle instead of an exploit cycle | 0.5 |
| alpha | float | The fall of rate in the fitness evaluation | 0.1 |
| beta | float | Learning Rate for updating statistics | 0.2 |
| delta | float | The fraction of the mean fitness of the population below which the fitness of a classifier may be considered in its vote for deletion | 0.1 |
| e_0 | float | The error threshold under which accuracy of a classifier can be set to 1 | 10 |
| nu | float | Power parameter for fitness evaluation | 5 |
| theta_del | non negative integer | Specified the threshold over which the fitness of a classifier may be considered in its deletion probability | 20 |
| theta_GA | nonnegative float | The threshold for the GA application in an action set | 25 |
| theta_matching | non negative integer | Number of unique actions that must be represented in the match set (otherwise, covering) | the # of unique actions in dataset|
| theta_select | float from 0 - 1 | The fraction of the action set to be included in tournament selection | 0.5 |
| theta_sub | non negative integer | The experience of a classifier required to be a subsumer | 20 |
| init_e | float | The initial prediction error value when generating a new classifier (e.g in covering) | 0 |
| init_fitness | float | The initial prediction value when generating a new classifier (e.g in covering) | 0.01 |
| init_prediction | float | The initial prediction value when generating a new classifier (e.g in covering) | 10 |
| do_action_set_subsumption | boolean | Do subsumption in action set | False |
| do_GA_subsumption | boolean | Do subsumption in GA between parents and offspring | True |
| max_payoff | float | For single step problems, what the maximum reward for correctness | 1000 |
| fitness_reduction | float | The reduction of the fitness when generating an offspring classifier | 0.1 |
| prediction_error_reduction | float | The reduction of the prediction error when generating an offspring classifier | 0.25 |

Below, for your reference, we also put together a table that shows how the training process may be effected, if one of the above attributes were changed:

| Parameter Name | Impact on training when changed |
| :--- | :--- |
| learning_iterations | more iterations increases accuracy (up to a point - there exists diminishing returns), but also increases training time |
| N | for larger feature spaces, a larger N is recommended. A larger N increases training time, but too small of an N will prevent the LCS from converging on a solution |
| p_general | the more attributes in the data set, the higher p_general should be. Too high of a p_spec and the population initialization will be less suited to detecting complex patterns, but too low can run the risk of early overfitting |
| p_crossover | Typically between 0.5 and 1. Recommended to use default parameter |
| p_mutation | Typically between 0.01 and 0.05. Recommended to use default parameter |
| p_explore | Recommended to use default parameter |
| alpha | Recommended to use default parameter |
| beta | Typically between 0.1 and 0.2. Recommended to use default parameter |
| delta | Recommended to use default parameter |
| e_0 | Typically about 1 percent of max_payoff |
| nu | recommended to be 1 for data with any level of noise. Increasing nu in clean problems improves chances of converging on optimal solution. Default of 5 is common for clean problems |
| theta_del | Recommended to use default parameter |
| theta_GA | Typically between 25 to 50. Recommended to use default parameter|
| theta_matching | Recommended to use default parameter |
| theta_select | Recommended to use default parameter |
| theta_sub | Recommended to use default parameter |
| init_e | Recommended to use default parameter |
| init_fitness | Recommended to use default parameter |
| init_prediction | Recommended to use default parameter |
| do_action_set_subsumption | Is stronger of the two subsumptions. Subsumption is useful in problems where this is a well-defined underlying target function. |
| do_GA_subsumption | Is weaker of the two subsumptions. Subsumption is useful in problems where this is a well-defined underlying target function. |
| max_payoff | Arbitrary, but 1000 is commonly used |
| fitness_reduction | Recommended to use default parameter |
| prediction_error_reduction | Recommended to use default parameter |

There also exists a few hyperparameters related to the setup and evaluation of the training process:

| Parameter Name | Requirements | Description | Default Value |
| :--- | :--- | :--- | :--- |
| discrete_attribute_limit | non negative integer OR "c" OR "d" | Multipurpose param. If it is a nonnegative integer, discrete_attribute_limit determines the threshold that determines if an attribute will be treated as a continuous or discrete attribute. For example, if discrete_attribute_limit == 10, if an attribute has more than 10 unique values in the dataset, the attribute will be continuous. If the attribute has 10 or less unique values, it will be discrete. Alternatively, discrete_attribute_limit can take the value of "c" or "d". See next param for this. | 10 |
| specified_attributes | numpy.ndarray of nonnegative integers of attribute indices | If discrete_attribute_limit == "c", attributes specified by index in this param will be continuous and the rest will be discrete. If "d", attributes specified by index in this param will be discrete and the rest will be continuous | empty numpy ndarray |
| random_state | integer OR None | Set a constant random seed value to some integer (in order to obtain reproducible results). Put None if none (pseudo-random algorithm runs) | None |
| reboot_filename | str or None | File name of pickled model to be rebooted. None by default (no model to reboot) | None |


These hyperparameters can be set during initialization. There exists built in parameter checking to ensure each specified parameter is valid. Below is an example:

In [8]:
model = XCS(learning_iterations = 5000,nu = 10)

## Part 3: Model Training and Testing

### model.fit(X, y) and how it works
Once the model is initialized with the hyperparameters you want, you can train the model by calling the fit(X,y) method, where X is the numpy.ndarray of attributes and y is the numpy.ndarray of phenotypes. Be sure both X and y satisfy the requirements specified in Part 1, or the parameter checker will raise an Exception. Below we demo the entire process of importing relevant packages, loading data, model initialization, and model training:

In [10]:
from skXCS import StringEnumerator
from skXCS import XCS
import numpy as np

#Use StringEnumerator to gather data
converter = StringEnumerator("test/DataSets/Real/Multiplexer11.csv","class")
headers,actionLabel,dataFeatures,dataActions = converter.get_params()

#Initialize and train model
model = XCS(learning_iterations = 5000)
trainedModel = model.fit(dataFeatures,dataActions)

trainedModel

TypeError: match() missing 1 required positional argument: 'xcs'

In [ ]:
trainedModel.env.formatData.trainFormatted[0]

The fit function returns a trained model. Below we will provide a high level overview of the scikit-XCS training algorithm. In compiling this algorithm, we aimed to make scikit-XCS as close to a 'modern' XCS as possible.

<ul>
    <li> Before training, scikit-XCS uses the training set to discern each attribute as continuous or discrete based on the <b>discrete_attribute_limit</b> and <b>specified_attributes</b> hyperparameters. It also counts the number of unique actions and determines if the problem is multiclass or binary. The training set is shuffled, and training begins. </li>
    <li> At the start of each iteration, an unlabelled training instance is fetched from the environment. </li>
    <li> Using the <b>p_explore</b> hyperparameter, scikit-XCS probabilistically determines if the learning iteration should be an 'explore' or 'exploit' iteration.</li>
    <li> A match set is created. For multiclass problems, coverage is by default triggered until all actions are represented in [M] (unless the <b>theta_matching</b> hyperparameter was changed). For binary problem coverage, it is the same, but coverage also triggers (once) if there are less than 5 classifiers in [M] </li>
    <li> If the iteration is 'explore', a random action in the match set is chosen. If the iteration is 'exploit', the classifiers in the match set each cast a weighted vote based on prediction and fitness to select the 'best action' </li>
    <li> The chosen action is given to the environment for execution. Since it is a single step classification problem, the environment is very simple. If the chosen action matches with the correct action for that instance, the environment returns the maximum reward. If the chosen action does not match, a reward of 0 is returned. </li>
    <li> Based on the reward, the experience, prediction, prediction error, and fitness of the classifiers in [A] are updated. If the <b>do_action_set_subsumption</b> hyperparameter is True, subsumption is done in [A] after this. </li>
    <li> If the iteration is 'explore', GA can occur. GA then occurs if the average number of iterations since the last GA is greater than hyperparameter <b>theta_GA</b> for classifiers in [A]. Tournament selection is used to choose two parent classifiers. Mutation and uniform crossover are then applied to create offspring classifiers. If the <b>do_GA_subsumption</b> hyperparameter is True, subsumption is done between the offspring and the parents before the offspring are added back into [P]. </li>
    <li> If the micropopulation (sum of numerosities) is greater than hyperparameter <b>N</b>, deletion occurs until it is. </li>
    <li> Training ends when the number of iterations equals the hyperparameter <b>learning_iterations</b>.
</ul>

### model.predict(X), model.predict_proba(X)

Once the model has finished training, you can call the predict(X) method to return an numpy.ndarray of action predictions given an array of unlabelled instances. The predict method works by using the final rule population of the model to vote on the most probable action based on classifier fitness and numerosity. Before making a prediction, parameter checkers again ensure X is a valid input (fully numeric numpy.ndarray). We refer back to the trained model above.

In [ ]:
trainedModel.predict(dataFeatures)

You can also call the predict_proba(X) method to return an nxm numpy.ndarray of phenotype probabilities, where n = number of data instances and m = number of distinct discrete phenotypes.

In [ ]:
trainedModel.predict_proba(dataFeatures)

### Cross Validation and Scoring
You can score a set of predictions using the score(X,y) method, where X is the unlabelled test data and y is associated list of correct action values. By default, score(X,y) uses balanced accuracy to score.

In [ ]:
trainedModel.score(dataFeatures,dataActions)

You can also use different scoring metrics to score the model. We will do this with a quick cross validation on the model. Below we will run a 3 fold CV on the trained model from above. Again, by default, balanced accuracy is used to score each partition.

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

#Shuffle Data (useful for cross validation)
formatted = np.insert(dataFeatures,dataFeatures.shape[1],dataActions,1)
np.random.shuffle(formatted)
dataFeatures = np.delete(formatted,-1,axis=1)
dataActions = formatted[:,-1]

np.mean(cross_val_score(trainedModel,dataFeatures,dataActions,cv=3))

If you prefer to use a scoring method other than balanced accuracy during CV (e.g. accuracy_score, precision_score, recall_score, f1_score), that can be specified as well via the standard scoring parameter. Be aware that some of these scoring methods only accept binary endpoints.

In [ ]:
np.mean(cross_val_score(trainedModel,dataFeatures,dataActions,cv=3,scoring="recall"))

Below we demonstrate a scoring method that calls predict_prob(X):

In [ ]:
np.mean(cross_val_score(trainedModel,dataFeatures,dataActions,cv=3,scoring="roc_auc"))

### ROC/PRC curves

As shown above, you can compute the AUC score of the ROC using the standard scikit scoring methods/metrics. Below, we show a means to graph the ROC and PRC, and compute the AUC for both curves (on training data). Again, be aware that ROC/PRC analysis only accepts binary endpoints.

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

probs = trainedModel.predict_proba(dataFeatures)
fpr, tpr, thresholds = roc_curve(dataActions, probs[:, 1])
prec, recall, thresholds = precision_recall_curve(dataActions, probs[:, 1])

plt.plot(fpr,tpr,label="ROC")
plt.plot(recall,prec,label="PRC")

plt.xlabel('False Positive Rate/Recall')
plt.ylabel('True Positive Rate/Precision')

plt.legend()
plt.show()

print("PRC AUC:" + str(auc(recall, prec)))
print("ROC AUC:" + str(auc(fpr, tpr)))


### Auxilliary scikit-XCS methods
In addition to the standard sklearn methods you can use to access training metrics, scikit-XCS provides a few of its own.

| Method Name | Description |
| :--- | :--- |
| model.get_final_training_accuracy() | Evaluates (in real time) the final training accuracy of the model |
| model.get_final_instance_coverage() | Evaluates (in real time) the final instance coverage of the model |
| model.get_final_attribute_specificity_list() | Returns a list of attribute specificities (which attributes are specified the most in the rule population) |
| model.get_final_attribute_accuracy_list() | Returns a list of attribute specificities weighted by accuracy |

In [ ]:
print("Final Training Accuracy: "+str(trainedModel.get_final_training_accuracy()))
print("Final Instance Coverage: "+str(trainedModel.get_final_instance_coverage()))
print("Final Attribute Specificity List: "+str(trainedModel.get_final_attribute_specificity_list()))
print("Final Attribute Accuracy List: "+str(trainedModel.get_final_attribute_accuracy_list()))

## Part 4: Data Collection and Export

### Exporting and Accessing Iteration Tracking Data
During training, scikit-XCS tracks a series of training statistics that may be of use. Aside from average rule population generality (which is tracked at least once an epoch), these following statistics are tracked every iteration:
<ul>
    <li>Approximate Training Accuracy (50 pt moving average)</li>
    <li>Macropopulation size</li>
    <li>Micropopulation size</li>
    <li>Match set size</li>
    <li>Action set size</li>
    <li>Average iteration age of classifiers in action set</li>
    <li>Number of classifiers subsumed in iteration</li>
    <li>Number of crossover operations performed in iteration (GA operation)</li>
    <li>Number of mutation operations performed in iteration (GA operation)</li>
    <li>Number of covering operations performed in iteration</li>
    <li>Number of macroclassifiers deleted in iteration</li>
    <li>Total training time</li>
    <li>Total matching time</li>
    <li>Total deletion time</li>
    <li>Total subsumption time</li>
    <li>Total selection time</li>
    <li>Total evaluation time</li>
</ul>
These statistics are not time intensive to compute, and thus can be tracked every iteration. 

Why is collecting this kind of iteration by iteration data important? Unlike many other machine learning models, as the XCS trains, it is easy to visualize its training process, such as how its accuracy or its macropopulation size changes over time. Thus, doing iteration tracking will allow you to export this training progess data (as you will see below) to aid in this visualization. Also, LCSs are stochastic algorithms with no clear stop criteria, so tracking these metrics will allow you to identify when training has stagnated, which can help in future hyperparameter optimization.

To access this iteration tracking data, you can call the following method that exports the entire tracking data record from training into a local folder:

| Method Name | Description |
| :--- | :--- |
| model.export_iteration_tracking_data(filename) | Exports all tracked data from training |

The filename paramater is optional. By default, the program saves a CSV file named **iterationData.csv** into your local directory. However, you can set the filename param as an absolute path with the format **/filepath/filename.csv** which will save the data into any specified directory.

In [ ]:
trainedModel.export_iteration_tracking_data("defaultExportDir/iterationData.csv")

The exported CSV is below:

In [ ]:
iterationData = pd.read_csv("defaultExportDir/iterationData.csv")
display(iterationData)

Once the data is exported to a CSV, you can plot trendlines for each statistic. We will use pandas and matplotlib as a demo below. Please note that for the average accuracy and average generality graphs, the first number of iterations is not as "accurate" as the rest of the iterations. The average accuracy moving average only begins moving at iteration 50, and average generality is not tracked until a few hundred iterations in.

In [ ]:
import matplotlib.pyplot as plt

def cumulativeFreq(freq):
    a = []
    c = []
    for i in freq:
        a.append(i+sum(c))
        c.append(i)
    return np.array(a)

def movingAvg(a,threshold=300):
    weights = np.repeat(1.0,threshold)/threshold
    conv = np.convolve(a,weights,'valid')
    return np.append(conv,np.full(threshold-1,conv[conv.size-1]),)

dataTracking = pd.read_csv("defaultExportDir/iterationData.csv")

iterations = dataTracking["Iteration"].values
accuracy = dataTracking['Accuracy (approx)'].values
generality = dataTracking['Average Population Generality'].values
macroPop = dataTracking["Macropopulation Size"].values
microPop = dataTracking["Micropopulation Size"].values
mSize = dataTracking["Match Set Size"].values
aSize = dataTracking["Action Set Size"].values
experience = dataTracking["Average Iteration Age of Action Set Classifiers"].values
subsumption = dataTracking["# Classifiers Subsumed in Iteration"].values
crossover = dataTracking["# Crossover Operations Performed in Iteration"].values
mutation = dataTracking["# Mutation Operations Performed in Iteration"].values
covering = dataTracking["# Covering Operations Performed in Iteration"].values
deletion = dataTracking["# Deletion Operations Performed in Iteration"].values

gTime = dataTracking["Total Global Time"].values
mTime = dataTracking["Total Matching Time"].values
delTime = dataTracking["Total Deletion Time"].values
subTime = dataTracking["Total Subsumption Time"].values
selTime = dataTracking["Total GA Time"].values
evalTime = dataTracking["Total Evaluation Time"].values

plt.plot(iterations,accuracy,label="approx accuracy")
plt.plot(iterations,generality,label="avg generality")
plt.xlabel('Iteration')
plt.ylabel('accuracy/generality')
plt.legend()
plt.show()

plt.plot(iterations,macroPop,label="macroPop Size")
plt.plot(iterations,microPop,label="microPop Size")
plt.xlabel('Iteration')
plt.ylabel('Macro/MicroPop Size')
plt.legend()
plt.show()

plt.plot(iterations,mSize,label="[M] size")
plt.plot(iterations,movingAvg(mSize),label="[M] size movingAvg")
plt.plot(iterations,aSize,label="[A] size")
plt.plot(iterations,movingAvg(aSize),label="[A] size movingAvg")
plt.xlabel('Iteration')
plt.ylabel('[M]/[A] size per iteration')
plt.legend()
plt.show()

plt.plot(iterations,experience)
plt.ylabel('Average [A] Classifier Age')
plt.xlabel('Iteration')
plt.show()

plt.plot(iterations,cumulativeFreq(subsumption),label="Subsumption Count")
plt.plot(iterations,cumulativeFreq(crossover),label="Crossover Count")
plt.plot(iterations,cumulativeFreq(mutation),label="Mutation Count")
plt.plot(iterations,cumulativeFreq(deletion),label="Deletion Count")
plt.plot(iterations,cumulativeFreq(covering),label="Covering Count")
plt.xlabel('Iteration')
plt.ylabel('Cumulative Operations Count Over Iterations')
plt.legend()
plt.show()

plt.plot(iterations,mTime,label="Matching Time")
plt.plot(iterations,delTime+mTime,label="Deletion Time")
plt.plot(iterations,subTime+delTime+mTime,label="Subsumption Time")
plt.plot(iterations,selTime+subTime+delTime+mTime,label="GA Time")
plt.plot(iterations,evalTime+selTime+subTime+delTime+mTime,label="Evaluation Time")
plt.plot(iterations,gTime,label="Total Time")
plt.xlabel('Iteration')
plt.ylabel('Cumulative Time (Stacked)')
plt.legend()
plt.show()

### Exporting and Accessing Rule Population Data
Post-training, you also have the option to export the final rule population. This can be done be calling one of the following methods:

| Method Name | Description |
| :--- | :--- |
| model.export_final_rule_population(filename,headerNames,classLabel) | Exports the entire final rule population with traditional rule representation|
| model.export_final_rule_population_DCAL(filename,headerNames,classLabel) | Exports the entire final rule population with DCAL rule representation|

The headerNames and classLabel params are optional. If they are not provided, default header and class names will be populated. But they make the exported csv more descriptive.

The filename param is also optional. By default, the program saves a CSV file named **populationData.csv** into your local directory. However, you can set the filename param as an absolute path with the format **/filepath/filename.csv** which will save the data into any specified directory.

In [ ]:
trainedModel.export_final_rule_population("defaultExportDir/fileRulePopulation.csv",headers,classLabel)

populationData2 = pd.read_csv("defaultExportDir/fileRulePopulation.csv")
display(populationData2)

And the above final rule population exported in DCAL format:

In [ ]:
trainedModel.export_final_rule_population_DCAL("defaultExportDir/popData2.csv",headers,classLabel)

populationData2 = pd.read_csv("defaultExportDir/popData2.csv")
display(populationData2)

## Part 5: Population Reboot
The final feature to introduce is the ability to "reboot" past rule populations. In some cases, you might want to run training, pause training, and resume training at some point in the future. Alternatively, you might want to train the generated rule population on new datasets or with different hyperparameters. The population reboot feature allows you to "save" the current rule population into a txt file for future use. You can later initialize a new XCS estimator with this txt file. When you run fit, the new XCS estimator will effectively start off where the saved model left off, beginning with the saved rule population, as well as starting at the next sequential learning iteration and adding to the last model's training times. A demo of this is shown below.

In [ ]:
converter = StringEnumerator("test/DataSets/Real/Multiplexer11.csv","class")
headers,classLabel,dataFeatures,dataActions = converter.get_params()

#Run 1
model = XCS(learning_iterations=1000)
model.fit(dataFeatures,dataActions)
print(model.score(dataFeatures,dataActions))
model.pickle_model("defaultExportDir/savedModel1")

#Run 2 w/ rebooted first model
model2 = XCS(learning_iterations=1000,reboot_filename="defaultExportDir/savedModel1")
print(model2.score(dataFeatures,dataActions))
model2.fit(dataFeatures,dataActions)
print(model2.score(dataFeatures,dataActions))
model2.pickle_model("defaultExportDir/savedModel2")

#Run 3 w/ rebooted second model
model3 = XCS(learning_iterations=1000,reboot_filename="defaultExportDir/savedModel2")
print(model3.score(dataFeatures,dataActions))
model3.fit(dataFeatures,dataActions)
print(model3.score(dataFeatures,dataActions))
model3.pickle_model("defaultExportDir/savedModel3")

#Run 4 w/ rebooted third model
model4 = XCS(learning_iterations=1000,reboot_filename="defaultExportDir/savedModel3")
print(model4.score(dataFeatures,dataActions))
model4.fit(dataFeatures,dataActions)
print(model4.score(dataFeatures,dataActions))
